In [1]:
# 作業：資料準備區
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
%matplotlib inline

#顯示所有行
pd.set_option('display.max_columns', None)
#顯示所有列
pd.set_option('display.max_rows', None)
#顯示行設定
pd.set_option('max_colwidth',100)

import pingouin as pg
import researchpy

df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

#1.產生一個新的變數 Survived_cate，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

# 取出資料後，把遺失值刪除
complete_data=df_train[['Age','Survived_cate','Sex']].dropna()
display(complete_data.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1

,Age,Survived_cate,Sex
0,22.0,0,male
1,38.0,1,female
2,26.0,1,female
3,35.0,1,female
4,35.0,0,male


In [2]:
# Q1: 產生一個新的變數(Ageabove65) Age>=65為 'Y'，其餘為'N'。
# 運用 function  做變數轉換
def age_check(x): 
    if(x>=65):
        return('Y')
    else: 
        return('N')

complete_data['Ageabove65']=complete_data['Age'].apply(age_check)
print('Q1答案：產生一個新的變數(Ageabove65)資料，Age>=65為 Y，其餘為 N，如下表顯示：')
display(complete_data.head(5)) # 太多資料 Github 有時容易 timeout

Q1答案：產生一個新的變數(Ageabove65)資料，Age>=65為 Y，其餘為 N，如下表顯示：


,Age,Survived_cate,Sex,Ageabove65
0,22.0,0,male,N
1,38.0,1,female,N
2,26.0,1,female,N
3,35.0,1,female,N
4,35.0,0,male,N


In [3]:
# Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。
# 再運用一個 function  做變數轉換
def femaleOrabove65(complete_data): 
    if complete_data['Ageabove65'] == 'Y' or complete_data['Sex'] == 'female':
        return 'Y'
    else:
        return 'N'

complete_data['Age_above65_female']=complete_data.apply(femaleOrabove65, axis=1)
print('Q2答案：產生一個新的變數(Age_above65_female)，女性或Age>=65為 Y，其餘為 N，如下表顯示：')
display(complete_data.head(5)) 

Q2答案：產生一個新的變數(Age_above65_female)，女性或Age>=65為 Y，其餘為 N，如下表顯示：


,Age,Survived_cate,Sex,Ageabove65,Age_above65_female
0,22.0,0,male,N,N
1,38.0,1,female,N,Y
2,26.0,1,female,N,Y
3,35.0,1,female,N,Y
4,35.0,0,male,N,N


In [4]:
# Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高
# 判斷為離散型採 Cramér's V
# 寫一個 function 判斷相關性的強度
def judgment_CramerV(df,V):
    if df == 1:
        if V < 0.10:
            qual = 'negligible'
        elif V < 0.30:
            qual = 'small'
        elif V < 0.50:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 2:
        if V < 0.07:
            qual = 'negligible'
        elif V < 0.21:
            qual = 'small'
        elif V < 0.35:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 3:
        if V < 0.06:
            qual = 'negligible'
        elif V < 0.17:
            qual = 'small'
        elif V < 0.29:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 4:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.15:
            qual = 'small'
        elif V < 0.25:
            qual = 'medium'
        else:
            qual = 'large'
    else:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.13:
            qual = 'small'
        elif V < 0.22:
            qual = 'medium'
        else:
            qual = 'large'
    return(qual)

# 用交叉列聯表(contingency table)
contTB65 = pd.crosstab(complete_data['Survived_cate'], complete_data['Ageabove65'])
print('交叉列聯表-年紀大於65歲:\n', contTB65)
contTB65F = pd.crosstab(complete_data['Survived_cate'], complete_data['Age_above65_female'])
print('\n交叉列聯表-女性或年紀大於65歲:\n', contTB65F)
# 計算自由度 df65 & df65F
df65 = min(contTB65.shape[0], contTB65.shape[1]) - 1
print('\n自由度-年紀大於65歲:', df65)
df65F = min(contTB65F.shape[0], contTB65F.shape[1]) - 1
print('自由度-女性或年紀大於65歲:', df65F)

crosstab, res = researchpy.crosstab(complete_data['Survived_cate'], complete_data['Ageabove65'], test='chi-square')
crosstabF, resF = researchpy.crosstab(complete_data['Survived_cate'], complete_data['Age_above65_female'], test='chi-square')

print("\nCramer's V-年紀大於65歲:",res.loc[2,'results'])
print("Cramer's V-女性或年紀大於65歲:",resF.loc[2,'results'])

print('\njudgment_CramerV-年紀大於65歲:', judgment_CramerV(df65,res.loc[2,'results']))
print('judgment_CramerV-女性或年紀大於65歲:', judgment_CramerV(df65F,resF.loc[2,'results']))
print('\n***************************************************')
print('\nQ3答案：透過數值法計算Age_above65_female(女性或年紀大於65歲) 和目標變數(Survived_cate)有較大關聯性。')

交叉列聯表-年紀大於65歲:
 Ageabove65       N   Y
Survived_cate         
0              414  10
1              289   1

交叉列聯表-女性或年紀大於65歲:
 Age_above65_female    N    Y
Survived_cate               
0                   350   74
1                    92  198

自由度-年紀大於65歲: 1
自由度-女性或年紀大於65歲: 1

Cramer's V-年紀大於65歲: 0.0803
Cramer's V-女性或年紀大於65歲: 0.514

judgment_CramerV-年紀大於65歲: negligible
judgment_CramerV-女性或年紀大於65歲: large

***************************************************

Q3答案：透過數值法計算Age_above65_female(女性或年紀大於65歲) 和目標變數(Survived_cate)有較大關聯性。
